# Use AutoGen in Databricks with DBRX

![DBRX launch](https://www.databricks.com/en-blog-assets/static/2fe1a0af1ee0f6605024a810b604079c/dbrx-blog-header-optimized.png)

In March 2024, Databricks released [DBRX](https://www.databricks.com/blog/introducing-dbrx-new-state-art-open-llm), a general-purpose LLM that sets a new standard for open LLMs. While available as an open-source model on Hugging Face ( [databricks/dbrx-instruct](https://huggingface.co/databricks/dbrx-instruct/tree/main) and [databricks/dbrx-base](https://huggingface.co/databricks/dbrx-base) ), customers of Databricks can also tap into the [Foundation Model APIs](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html#query-a-chat-completion-model), which make DBRX available through an OpenAI-compatible, autoscaling REST API.

[Autogen](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat) is becoming a popular standard for agent creation. Built to support any "LLM as a service" that implements the OpenAI SDK, it can easily be extended to integrate with powerful open source models. 

This notebook will demonstrate a few basic examples of Autogen with DBRX, including the use of  `AssistantAgent`, `UserProxyAgent`, and `ConversableAgent`.

## Requirements
AutoGen must be installed on your Databricks cluster, and requires `Python>=3.8`. This example includes the `%pip` magic command to install: `%pip install pyautogen`, as well as other necessary libraries.

This code has been tested on: 
* [Serverless Notebooks](https://docs.databricks.com/en/compute/serverless.html) (in public preview as of Apr 18, 2024)
* Databricks Runtime 14.3 LTS ML [docs](https://docs.databricks.com/en/release-notes/runtime/14.3lts-ml.html)

This code can run in any Databricks workspace in a region where DBRX is available via pay-per-token APIs (or provisioned throughput). To check if your region is supported, see [Foundation Model Region Availability](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-limits.html#foundation-model-apis-limits)

## Tips
* This notebook can be imported to a Databricks workspace and run directly. Use [sparse checkout mode with git](https://www.databricks.com/blog/2023/01/26/work-large-monorepos-sparse-checkout-support-databricks-repos.html) to import only this notebook or the examples directory. 

* Databricks recommends using [Secrets](https://docs.databricks.com/en/security/secrets/secrets.html) instead of storing tokens in plain text. 

In [0]:
%pip install pyautogen==0.2.25 openai==1.21.2 typing_extensions==4.11.0 --upgrade
dbutils.library.restartPython()

## Setup DBRX config list

See Autogen docs for more inforation on the use of `config_list`: [LLM Configuration](https://microsoft.github.io/autogen/docs/topics/llm_configuration#why-is-it-a-list)

In [0]:
import os

# Set environment variables with your current workspace host and a personal access token
# To a secret you have already set up: dbutils.secrets.get('your_scope_name','databricks_host')

# DATABRICKS_HOST format: "https://{your workspace url}" (no trailing slash)
## AWS Workspace example: "https://my-databricks-workspace.cloud.databricks.com"
## Azure Workspace example: "https://adb-123456790123.12.azuredatabricks.net"
os.environ['DATABRICKS_HOST'] = "<FILL IN WITH YOUR WORKSPACE URL IN SUPPORTED DBRX REGION>"

# DATABRICKS_TOKEN format: "dapi..."
os.environ['DATABRICKS_TOKEN'] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

llm_config = {
    "config_list": [
      {
        "model": "databricks-dbrx-instruct", 
        "api_key": str(os.environ["DATABRICKS_TOKEN"]),
        "base_url": str(os.getenv("DATABRICKS_HOST"))+"/serving-endpoints"
        }
      ],
}

## Hello World Example

Our first example will be with a simple `UserProxyAgent` asking a question to an `AssistantAgent`. This is based on the tutorial demo [here](https://microsoft.github.io/autogen/docs/tutorial/introduction).

After sending the question and seeing a response, you can type `exit` to end the chat or continue to converse.

In [0]:
import autogen

# Create Assistant and User 
assistant = autogen.AssistantAgent(
  name="assistant", 
  llm_config=llm_config
  )

user_proxy = autogen.UserProxyAgent(
  name="user", 
  code_execution_config=False
  )

# Initiate chat from user_proxy side
user_proxy.initiate_chat(
  assistant, 
  message="What is MLflow?"
  )

## Simple Coding Agent

In this example, we will implement a "coding agent" that can execute code. You will see how this code is run alongside your notebook in your current workspace, taking advantage of the performance benefits of Databricks clusters. This is based off the demo [here](https://microsoft.github.io/autogen/docs/topics/non-openai-models/cloud-mistralai/).

First, set up a directory: 

In [0]:
from pathlib import Path
workdir = Path("coding")
print(workdir)
workdir.mkdir(exist_ok=True)

from autogen.coding import LocalCommandLineCodeExecutor
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

Next, setup our agents and initiate a coding problem. Notice how the `UserProxyAgent` will take advantage of our `code_executor`; after the code is shown on screen, type Return/Enter in the chatbox to have it execute locally on your cluster via the bot's auto-reply. Note: with generative AI coding assistants, you should **always** manually read and review the code before executing it yourself:

In [0]:
from autogen import AssistantAgent, UserProxyAgent
# Setting up the agents.
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content"),
)

assistant_agent = AssistantAgent(
    name="DBRX Assistant",
    llm_config=llm_config,
)

chat_result = user_proxy_agent.initiate_chat(
    assistant_agent,
    message="Count how many prime numbers from 1 to 10000.",
)

We can see the python file that was created in our working directory:

In [0]:
%sh ls coding

In [0]:
%sh head coding/count_primes.py

## Conversable Bots

We can also implement the [two-agent chat pattern](https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns/#two-agent-chat-and-chat-result) using DBRX to "talk to itself" in a teacher/student exchange:

In [0]:
from autogen import ConversableAgent

# Setting up the agents.
student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config=llm_config,
)

teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a computer science teacher.",
    llm_config=llm_config,
)

# Initiate chat
chat_result = student_agent.initiate_chat(
        teacher_agent,
        message="How does deep learning relate to artificial intelligence?",
        summary_method="reflection_with_llm",
        max_turns=1, #Set to higher number to control back and forth
    )

## Implement Logging Display

It can be useful to display chat logs to the notebook for debugging, and then persist those logs to a Delta table. The following section demonstrates how to extend the default AutoGen logging libraries.

First, we will implement a Python `class` that extends the capabilities of `autogen.runtime_logging`:

In [0]:
class Databricks_AutoGenLogger():
  def __init__(self):
    import autogen
    self.logger_config = {"dbname": "logs.db"}

  def start(self):
    import autogen.runtime_logging
    self.logging_session_id = autogen.runtime_logging.start(config=self.logger_config)
    print("Logging session ID: " + str(self.logging_session_id))

  def stop(self):
    import autogen.runtime_logging
    autogen.runtime_logging.stop()

  def _get_log(self, dbname="logs.db", table="chat_completions"):
    import sqlite3
    con = sqlite3.connect(dbname)
    query = f"SELECT * from {table} WHERE session_id == '{self.logging_session_id}' ORDER BY end_time DESC"
    cursor = con.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    data = [dict(zip(column_names, row)) for row in rows]
    con.close()
    return data

  def display_session(self):
    import pandas as pd
    return pd.DataFrame(self._get_log())
  
  def persist_results(self, target_delta_table:str, mode="append"):
    import pandas as pd
    # Convert to Spark DF
    sdf = spark.createDataFrame(pd.DataFrame(self._get_log()))

    try: 
      sdf.write.format("delta").mode(mode).saveAsTable(target_delta_table)
      print(f"Logs sucessfully written to table {target_delta_table} in {mode} mode")
    except Exception as e:
      print(f"An error occurred: {e}")


Let's use the class above on our simplest example:

In [0]:
assistant = autogen.AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = autogen.UserProxyAgent(name="user", code_execution_config=False)

# Before initiating chat, start logging:
logs = Databricks_AutoGenLogger()
logs.start()
try:
  user_proxy.initiate_chat(assistant, message="What is MLflow?", max_turns=1)
except:
  print("An Error occurred")
logs.stop()
# Display logs
display(logs.display_session())

With this, we have a simple framework to review and persist logs from our chats! Notice that in the `request` field above, we can also see the system prompt for the LLM - this can be useful for prompt engineering as well as debugging.

Note that when you deploy this to Databricks Model Serving, model responses are auto-logged using [Lakehouse Monitoring](https://docs.databricks.com/en/lakehouse-monitoring/index.html); but the above approach provides a simple mechanism to log chats from the **client side**.

Let's now persist these results to a Delta table in [Unity Catalog](https://docs.databricks.com/en/data-governance/unity-catalog/index.html):

In [0]:
# Use 3-layer namespace: catalog.schema.table. The table will be created if it does not exist.
target_delta_table = "your_catalog.your_schema.autogen_logs"
logs.persist_results(target_delta_table=target_delta_table, mode="append")

display(spark.table(target_delta_table))

## Closing Thoughts
This notebook provides a few basic examples of using Autogen with DBRX, and we're excited to see how you can use this framework alongside leading open-source LLMs!

### Limitations
* Databricks Foundation Model API supports other open-source LLMs (Mixtral, Llama2, etc.), but the above code has not been tested on those.

* As of April 2024, DBRX does not yet support tool/function calling abilities. To discuss this capability further, please reach out to your Databricks representative.